In [1]:
import pandas as pd

In [2]:
odds_file = 'OddsAPI.csv'
bye_file = 'ByeWeeks.csv'
data = pd.read_csv(odds_file)
byes = pd.read_csv(bye_file).set_index('Team')['Bye'].to_dict()
data['gameid'] = data['event_name'] + data['commence']

In [3]:
def parse_game(game):
    team1, team2 = game['event_name'].iloc[0].split('_')
    odd1 = game['odd_1'].mean()
    odd2 = game['odd_2'].mean()
    p1 = 1 / (1 + odd1 / odd2)
    p2 = 1 / (1 + odd2 / odd1)
    return (team1, p1, team2, p2)

In [4]:
df = None

for gid in data['gameid'].unique():
    game = data[data['gameid'] == gid]
    team1, p1, team2, p2 = parse_game(game)
    new_df = pd.DataFrame([[team1, p1], [team2, p2]], columns=['Team', 'Prob'])
    
    if df is None:
        df = new_df
    else:
        df = pd.concat([df, new_df])


In [5]:
probs_df = None
cols = ['Team'] + [f'Week{i + 1}' for i in range(18)]

for team, bye in byes.items():
    team_df = df[df['Team'] == team].reset_index()
    
    team_probs = [team]
    bye_flag = False
    for i in range(18):
        try:
            if i + 1 == bye:
                team_probs.append(0.0)
                bye_flag = True
            elif not bye_flag:
                team_probs.append(team_df.iloc[i]['Prob'])
            else:
                team_probs.append(team_df.iloc[i - 1]['Prob'])
        except IndexError:
            team_probs.append(0)
    new_df = pd.DataFrame(data=[team_probs], columns=cols)
    if probs_df is None:
        probs_df = new_df
    else:
        probs_df = pd.concat([probs_df, new_df])

In [6]:
probs_df.to_csv('ProbMatrix.csv', index=False)